# NLST Predictions Merge

Match and put together NLST predictions for Kiran, Thijmen, PanCan and Sybil (after those predictions have ran).

In [1]:
import pandas as pd
import os
import numpy as np

import seaborn as sns
sns.set_style("white")
from evalutils.roc import get_bootstrapped_roc_ci_curves
import matplotlib.pyplot as plt

%matplotlib inline
import sklearn.metrics as skl_metrics

## directory where results are
LOCAL_PC = False
root_dir = "/mnt/w" if LOCAL_PC else "/data/bodyct"
EXPERIMENT_DIR = f"{root_dir}/experiments/lung-malignancy-fairness-shaurya"
NLST_PREDS = f"{EXPERIMENT_DIR}/nlst-preds"

NLST_PREDS_LOCAL = "/mnt/c/Users/shaur/OneDrive - Radboudumc/Documents - Master - Shaurya Gaur/General/Malignancy-Estimation Results/nlst"
NLST_PREDS = NLST_PREDS_LOCAL ## Comment out if not using Teams backup (aka Chansey is up :)

## Load Predictions

Kiran's model and PanCan2b.

In [2]:
nlst_kiran_pancan = pd.read_csv(f"{NLST_PREDS}/NLST_DL_vs_PanCan_Venk21.csv")
nlst_kiran_pancan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16077 entries, 0 to 16076
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PatientID            16077 non-null  int64  
 1   StudyDate            16077 non-null  int64  
 2   SeriesInstanceUID    16077 non-null  object 
 3   CoordX               16077 non-null  float64
 4   CoordY               16077 non-null  float64
 5   CoordZ               16077 non-null  float64
 6   LesionID             16077 non-null  int64  
 7   NoduleType           16077 non-null  object 
 8   Spiculation          16077 non-null  bool   
 9   Diameter [mm]        16077 non-null  float64
 10  Age                  16077 non-null  int64  
 11  Gender               16077 non-null  int64  
 12  FamilyHistoryLungCa  16077 non-null  bool   
 13  Emphysema            16077 non-null  bool   
 14  NoduleInUpperLung    16077 non-null  bool   
 15  NoduleCounts         16077 non-null 

Thijmen's model. Note that it only has 3240 annotation predictions and not 16077.

In [4]:
nlst_thijmen = pd.read_csv(f"{NLST_PREDS}/NLST_Tijmen_results/merged_model_output.csv")
nlst_thijmen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3240 entries, 0 to 3239
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   AnnotationID  3240 non-null   object 
 1   label         3240 non-null   float64
 2   Thijmen_mean  3240 non-null   float64
dtypes: float64(2), object(1)
memory usage: 76.1+ KB


Sanity check to make sure the labels match for the `AnnotationID` between Thijmen's CSV and the Kiran PanCan sheet. If the output DataFrame below is empty, you're all good :)

In [17]:
nlst_kp2 = nlst_kiran_pancan.sort_values(by='AnnotationID', ascending=True).set_index("AnnotationID")
nlst_tj2 = nlst_thijmen.sort_values(by='AnnotationID', ascending=True).set_index("AnnotationID")

nlst_kp2 = nlst_kp2[nlst_kp2.index.isin(nlst_tj2.index)]
res = nlst_kp2['label'].compare(nlst_tj2['label'])

del nlst_kp2, nlst_tj2
res

,self,other
AnnotationID,,


Merge the predictions!

In [23]:
nlst_preds = nlst_kiran_pancan.merge(nlst_thijmen, 
                                     how="left", ## 'inner' for only ones Thijmen has preds for
                                     on=['AnnotationID', 'label'], suffixes=(None,None))
nlst_preds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16077 entries, 0 to 16076
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PatientID            16077 non-null  int64  
 1   StudyDate            16077 non-null  int64  
 2   SeriesInstanceUID    16077 non-null  object 
 3   CoordX               16077 non-null  float64
 4   CoordY               16077 non-null  float64
 5   CoordZ               16077 non-null  float64
 6   LesionID             16077 non-null  int64  
 7   NoduleType           16077 non-null  object 
 8   Spiculation          16077 non-null  bool   
 9   Diameter [mm]        16077 non-null  float64
 10  Age                  16077 non-null  int64  
 11  Gender               16077 non-null  int64  
 12  FamilyHistoryLungCa  16077 non-null  bool   
 13  Emphysema            16077 non-null  bool   
 14  NoduleInUpperLung    16077 non-null  bool   
 15  NoduleCounts         16077 non-null 

Thijmen's doesn't seem like a size-matched subset with a higher malignant percentage.

In [26]:
nlst_thijmen_nonnull = nlst_preds[~nlst_preds['Thijmen_mean'].isnull()]
nlst_thijmen_nonnull.groupby('label').nunique()

,PatientID,StudyDate,SeriesInstanceUID,CoordX,CoordY,CoordZ,LesionID,NoduleType,Spiculation,Diameter [mm],...,NoduleCounts,SCT_EPI_LOC,xie_gc_gclobe150,loclup,locrup,PanCan2b,DL,NoduleID,AnnotationID,Thijmen_mean
label,,,,,,,,,,,,,,,,,,,,,
0,907,3,1797,2794,2834,2845,18,4,1,202,...,10,7,0,0,0,2722,2958,1825,2959,2959
1,154,3,265,279,280,280,2,3,2,159,...,8,6,6,2,2,280,281,164,281,281


## Load NLST Demographic Info

In [139]:
nlst_dict_full = pd.read_csv(f"{NLST_PREDS}/participant_d040722.csv")
nlst_dict_full.info()

/tmp/ipykernel_2342/2896154133.py:1: DtypeWarning: Columns (239,240,348) have mixed types. Specify dtype option on import or set low_memory=False.
  nlst_dict_full = pd.read_csv(f"{NLST_PREDS}/participant_d040722.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53452 entries, 0 to 53451
Columns: 356 entries, cen to evpsent
dtypes: float64(283), int64(64), object(9)
memory usage: 145.2+ MB


In [140]:
nlst_dict_subset = nlst_dict_full[nlst_dict_full['pid'].isin(nlst_preds['PatientID'])].reset_index(drop=True)
nlst_dict_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5282 entries, 0 to 5281
Columns: 356 entries, cen to evpsent
dtypes: float64(283), int64(64), object(9)
memory usage: 14.3+ MB


In [141]:
nlst_dict_subset.groupby(['gender'])['anyscr_has_nodule'].value_counts(dropna=False, normalize=True)

gender  anyscr_has_nodule
1       1.0                  0.986711
        0.0                  0.013289
2       1.0                  0.994278
        0.0                  0.005722
Name: proportion, dtype: float64

In [147]:
workfacets = ['res', 'wrk', 'yrs']
worktypes = ['asbe', 'baki', 'butc', 'chem', 'coal', 'cott', 'farm', 'fire', 'flou', 'foun', 'hard', 'pain', 'sand', 'weld']

diseasefacets = ['age', 'diag']
diseasetypes = ['adas', 'asbe', 'bron', 'chas', 'chro', 'copd', 'diab', 'emph', 'fibr', 'hear', 'hype', 'pneu', 'sarc', 'sili', 'stro', 'tube']

pcancerfacets = ['age', 'canc']
pcancertypes = ['blad', 'brea', 'cerv', 'colo', 'esop', 'kidn', 'lary', 'lung', 'nasa', 'oral', 'panc', 'phar', 'stom', 'thyr', 'tran']

alcohol_acrin = ['acrin_alc_curr', 'acrin_alc_ever', 'acrin_drink24hr', 'acrin_drinknum_curr', 'acrin_drinknum_form', 'acrin_drinkyrs_curr', 'acrin_drinkyrs_form', 'acrin_lastdrink']
alcohol_lss = ['lss_alcohol_freq', 'lss_alcohol_num']

cols = {
    "info": ['pid', 'study'],
    "demo": ['age', 'educat', 'ethnic', 'gender', 'height', 'marital', 'race', 'weight'],
    "smoking": ['age_quit', 'cigar', 'cigsmok', 'pipe', 'pkyr', 'smokeage', 'smokeday', 'smokelive', 'smokework', 'smokeyr'],
    "screen": ['scr_group'],
    "work": [facet+job for facet in workfacets for job in worktypes],
    "disease": [facet+disease for facet in diseasefacets for disease in diseasetypes],
    "personal_cancer": [facet+cancer for facet in pcancerfacets for cancer in pcancertypes],
    "family": ['fam'+member for member in ['brother', 'sister', 'child', 'father', 'mother']],
    "alcohol": alcohol_lss+alcohol_acrin,   
}

allcols = sum([cols[k] for k in cols], start=[])
len(allcols)

140

In [152]:
for c in cols:
    print(c, len(cols[c]))
    nlst_colinfo = nlst_dict_subset[cols[c]]
    display(nlst_colinfo.describe())

info 2


,pid,study
count,5282.000000,5282.000000
mean,148184.307081,1.474252
std,44395.945951,0.728020
min,100012.000000,1.000000
25%,113113.500000,1.000000
50%,125989.000000,1.000000
75%,205225.250000,2.000000
max,218887.000000,3.000000


demo 8


,age,educat,ethnic,gender,height,marital,race,weight
count,5282.000000,5282.000000,5282.000000,5282.000000,5273.000000,5281.000000,5282.000000,5270.000000
mean,62.183453,4.835290,2.538054,1.430140,67.846008,2.654043,1.307459,181.264706
std,5.184738,4.579593,7.283255,0.495142,3.921241,1.247796,3.831637,38.917809
min,54.000000,1.000000,1.000000,1.000000,49.000000,1.000000,1.000000,75.000000
25%,58.000000,3.000000,2.000000,1.000000,65.000000,2.000000,1.000000,154.000000
50%,61.000000,5.000000,2.000000,1.000000,68.000000,2.000000,1.000000,179.000000
75%,66.000000,6.000000,2.000000,2.000000,71.000000,3.000000,1.000000,205.000000
max,74.000000,99.000000,99.000000,2.000000,83.000000,9.000000,99.000000,368.000000


smoking 10


...

,ageblad,agebrea,agecerv,agecolo,ageesop,agekidn,agelary,agelung,agenasa,ageoral,...,canckidn,canclary,canclung,cancnasa,cancoral,cancpanc,cancphar,cancstom,cancthyr,canctran
count,35.000000,91.000000,78.000000,25.000000,5.000000,8.000000,5.000000,4.00,3.000000,14.000000,...,5276.000000,5276.000000,5276.000000,5274.000000,5275.000000,5276.0,5276.000000,5276.000000,5273.000000,5272.000000
mean,55.885714,49.626374,36.538462,53.840000,50.800000,54.625000,55.000000,65.25,62.000000,53.928571,...,0.001516,0.000948,0.000758,0.000569,0.002654,0.0,0.000190,0.001137,0.001517,0.000190
std,10.303544,8.452019,10.079801,8.979978,6.978539,7.424621,12.629331,4.50,6.244998,9.988180,...,0.038914,0.030773,0.027527,0.023846,0.051454,0.0,0.013767,0.033707,0.038925,0.013772
min,17.000000,25.000000,21.000000,32.000000,45.000000,48.000000,40.000000,59.00,55.000000,36.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,52.000000,43.500000,28.000000,50.000000,46.000000,49.500000,43.000000,63.50,59.500000,50.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
50%,56.000000,50.000000,34.500000,55.000000,48.000000,52.000000,60.000000,66.50,64.000000,54.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
75%,63.500000,55.500000,41.750000,59.000000,53.000000,57.500000,65.000000,68.25,65.500000,60.750000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
max,68.000000,68.000000,63.000000,67.000000,62.000000,67.000000,67.000000,69.00,67.000000,70.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000


family 5


,fambrother,famsister,famchild,famfather,fammother
count,5140.000000,5141.000000,5138.000000,5114.000000,5152.000000
mean,0.057977,0.042988,0.004866,0.109894,0.065800
std,0.233722,0.202849,0.069591,0.312789,0.247956
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


alcohol 10


,lss_alcohol_freq,lss_alcohol_num,acrin_alc_curr,acrin_alc_ever,acrin_drink24hr,acrin_drinknum_curr,acrin_drinknum_form,acrin_drinkyrs_curr,acrin_drinkyrs_form,acrin_lastdrink
count,3495.000000,2744.000000,1662.000000,1757.000000,1185.000000,1181.000000,404.000000,1174.000000,405.000000,435.000000
mean,2.931903,1.910350,1.737064,1.948776,0.972152,6.938188,13.975248,39.326235,23.711111,2.372414
std,1.459018,0.912094,0.440361,0.220516,1.622811,9.341564,20.147949,10.454303,13.555825,0.860914
min,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,35.000000,12.000000,1.000000
50%,3.000000,2.000000,2.000000,2.000000,0.000000,4.000000,5.000000,40.000000,23.000000,3.000000
75%,4.000000,2.000000,2.000000,2.000000,2.000000,10.000000,20.000000,45.000000,34.000000,3.000000
max,5.000000,5.000000,2.000000,2.000000,26.000000,89.000000,100.000000,70.000000,70.000000,3.000000


In [146]:
nlst_demos = nlst_dict_subset[allcols]
nlst_demos.describe()

,pid,study,age,educat,ethnic,gender,height,marital,race,weight,...,lss_alcohol_freq,lss_alcohol_num,acrin_alc_curr,acrin_alc_ever,acrin_drink24hr,acrin_drinknum_curr,acrin_drinknum_form,acrin_drinkyrs_curr,acrin_drinkyrs_form,acrin_lastdrink
count,5282.000000,5282.000000,5282.000000,5282.000000,5282.000000,5282.000000,5273.000000,5281.000000,5282.000000,5270.000000,...,3495.000000,2744.000000,1662.000000,1757.000000,1185.000000,1181.000000,404.000000,1174.000000,405.000000,435.000000
mean,148184.307081,1.474252,62.183453,4.835290,2.538054,1.430140,67.846008,2.654043,1.307459,181.264706,...,2.931903,1.910350,1.737064,1.948776,0.972152,6.938188,13.975248,39.326235,23.711111,2.372414
std,44395.945951,0.728020,5.184738,4.579593,7.283255,0.495142,3.921241,1.247796,3.831637,38.917809,...,1.459018,0.912094,0.440361,0.220516,1.622811,9.341564,20.147949,10.454303,13.555825,0.860914
min,100012.000000,1.000000,54.000000,1.000000,1.000000,1.000000,49.000000,1.000000,1.000000,75.000000,...,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,113113.500000,1.000000,58.000000,3.000000,2.000000,1.000000,65.000000,2.000000,1.000000,154.000000,...,2.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,35.000000,12.000000,1.000000
50%,125989.000000,1.000000,61.000000,5.000000,2.000000,1.000000,68.000000,2.000000,1.000000,179.000000,...,3.000000,2.000000,2.000000,2.000000,0.000000,4.000000,5.000000,40.000000,23.000000,3.000000
75%,205225.250000,2.000000,66.000000,6.000000,2.000000,2.000000,71.000000,3.000000,1.000000,205.000000,...,4.000000,2.000000,2.000000,2.000000,2.000000,10.000000,20.000000,45.000000,34.000000,3.000000
max,218887.000000,3.000000,74.000000,99.000000,99.000000,2.000000,83.000000,9.000000,99.000000,368.000000,...,5.000000,5.000000,2.000000,2.000000,26.000000,89.000000,100.000000,70.000000,70.000000,3.000000
